In [18]:
# 라이브러리 추출
import pandas as pd
import numpy as np
import tensorflow as tf

In [19]:
# 메서드 정의
def main(URL = 'D:\project\Teamproject1\JeseongMoon\Dataset\Regression_data.csv'):
    
    # 랜덤 시드 고정
    np.random.seed(42)
    tf.random.set_seed(42)
    
    global df, LEARNING_RATE, EPOCH_COUNT, MB_SIZE, REPORT, TRAIN_RATIO, X, y, X_train, X_test, y_train, y_test, y_pred
    
    df = load_dataset(URL) # 데이터 로드
    Regression_Model() # 회귀 모델

In [20]:
# 메서드 정의
def load_dataset(URL = 'D:\project\Teamproject1\JeseongMoon\Dataset\Regression_data.csv'):
    
    # 데이터셋 로드
    df = pd.read_csv(URL)

    # 성별 인코딩
    from sklearn.preprocessing import LabelEncoder
    encoder = LabelEncoder()
    df['Sex'] = encoder.fit_transform(df['Sex'])
    
    print(df.head())
    
    return df

In [21]:
# 사용자 정의 평가 지표 메서드(EvalAccuracy 클래스로 해당 기능을 구현했으며 참고용)
"""
def eval_accuracy(y_true, y_predict):
    
	# 오차율 구하는 과정    
	# np.mean() 메서드의 이유는 미니배치 처리를 고려하여 하나의 지표로 묶어주기 위함 입니다. 
    mdiff = np.mean(np.abs((y_predict - y_true) / y_true))
    # 1 에서 오차율을 빼 정확도를 구합니다. 
    return 1 - mdiff
"""

'\ndef eval_accuracy(y_true, y_predict):\n    \n\t# 오차율 구하는 과정    \n\t# np.mean() 메서드의 이유는 미니배치 처리를 고려하여 하나의 지표로 묶어주기 위함 입니다. \n    mdiff = np.mean(np.abs((y_predict - y_true) / y_true))\n    # 1 에서 오차율을 빼 정확도를 구합니다. \n    return 1 - mdiff\n'

In [22]:
# 사용자 정의 평가 지표 클래스
class EvalAccuracy(tf.keras.metrics.Metric): # TensorFlow의 Metric 클래스를 상속 받음

    def __init__(self, name="eval_accuracy", **kwargs): # 부모 클래스의 __init__() 메소드를 호출하여 필요한 초기화를 수행
        super(EvalAccuracy, self).__init__(name=name, **kwargs)
        self.correct = self.add_weight(name="ctp", initializer="zeros")
        # add_weight() 메소드를 사용하여 평가 지표를 계산하는데 필요한 변수를 생성(각 배치에서의 평가 결과를 누적하기 위해)
        # add_weight() 는 텐서플로우 Layer 클래스의 메서드(새로운 가중치를 추가하는 기능, 여기서는 평가 지표를 계산하는 데 사용되는 일종의 내부 변수를 의미)
        # 이 구문이 실행되면, EvalAccuracy 인스턴스는 새로운 가중치를 추가하고 그 가중치를 self.correct에 저장한다.
        # 이 self.correct는 update_state() 메서드에서 업데이트되며, '현재까지 처리한 모든 배치에 대한 평가 지표의 평균을 저장'한다.

    def update_state(self, y_true, y_predict, sample_weight=None):
        value = tf.abs((y_predict - y_true) / y_true)
        self.correct.assign(tf.reduce_mean(value)) # 오차율을 계산해서 correct 변수에 누적한 후, assign() 메소드를 사용하여 correct 변수의 값을 업데이트

    def result(self):
        return 1 - self.correct

    def reset_states(self):
        # 에포크마다 평가 지표 초기화
        self.correct.assign(0.)

In [23]:
def Regression_Model():
    # 학습 모델 구현
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import r2_score
    from sklearn.metrics import mean_squared_error

    # 상수 정의
    LEARNING_RATE = 0.01
    EPOCH_COUNT = 100
    MB_SIZE = 100
    REPORT = 1
    TRAIN_RATIO = 0.8

    # 학습 데이터 분리
    X = df.drop('Rings', axis=1)
    y = df['Rings']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=TRAIN_RATIO)
    
    # 텐서플로우의 텐서 연산은 계산 효율성 및 GPU사용을 최적화하기 위해 자동 형변환이 발생하지 않는다.
    # 따라서 매개변수를 float타입으로 변경해야 EvalAccuracy 클래스의 update_state 메서드가 작동한다. (사용자 정의 평가 지표)
    y_train = y_train.astype('float32')
    y_test = y_test.astype('float32')

    # 모델 생성
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=256, activation='relu', input_shape=(len(X_train.keys()),)), # (len(X_train.keys()),)로 입력해야 튜플 형태로 입력되어 오류가 발생하지 않음
        tf.keras.layers.Dense(units=128, activation='relu'),
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)
    ])


    # 옵티마이저와 손실 함수 설정
    optimizer = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE) # SGD : 경사하강법을 기본적으로 사용하는 옵티마이저
    model.compile(loss='mean_squared_error',
                  optimizer=optimizer,
                  metrics=[EvalAccuracy()])
    
    # 학습 시작
    model.fit(X_train, y_train, epochs=EPOCH_COUNT, batch_size=MB_SIZE, verbose=REPORT)
    
    # 모델 평가
    y_pred = model.predict(X_test)
    
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f'Accuracy: {accuracy}\n MSE: {loss}')

In [24]:
main()

   Sex  Length  Diameter  Height  Whole weight  Shucked weight  \
0    2   0.455     0.365   0.095        0.5140          0.2245   
1    2   0.350     0.265   0.090        0.2255          0.0995   
2    0   0.530     0.420   0.135        0.6770          0.2565   
3    2   0.440     0.365   0.125        0.5160          0.2155   
4    1   0.330     0.255   0.080        0.2050          0.0895   

   Viscera weight  Shell weight  Rings  
0          0.1010         0.150     15  
1          0.0485         0.070      7  
2          0.1415         0.210      9  
3          0.1140         0.155     10  
4          0.0395         0.055      7  
Epoch 1/100
34/34 [==============================] - 0s 1000us/step - loss: 42.4984 - eval_accuracy: 0.7234
Epoch 2/100
34/34 [==============================] - 0s 1ms/step - loss: 8.1504 - eval_accuracy: 0.6832
Epoch 3/100
34/34 [==============================] - 0s 970us/step - loss: 8.0429 - eval_accuracy: 0.7622
Epoch 4/100
34/34 [====================

c:\Users\82105\anaconda3\envs\project1\lib\site-packages\keras\metrics.py:257: UserWarning: Metric EvalAccuracy implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))


Epoch 7/100
34/34 [==============================] - 0s 1ms/step - loss: 7.2838 - eval_accuracy: 0.7715
Epoch 8/100
34/34 [==============================] - 0s 1ms/step - loss: 7.2965 - eval_accuracy: 0.8024
Epoch 9/100
34/34 [==============================] - 0s 1000us/step - loss: 7.5460 - eval_accuracy: 0.8114
Epoch 10/100
34/34 [==============================] - 0s 1ms/step - loss: 7.0663 - eval_accuracy: 0.7742
Epoch 11/100
34/34 [==============================] - 0s 1ms/step - loss: 7.0011 - eval_accuracy: 0.7138
Epoch 12/100
34/34 [==============================] - 0s 1000us/step - loss: 6.6871 - eval_accuracy: 0.8290
Epoch 13/100
34/34 [==============================] - 0s 1ms/step - loss: 6.8151 - eval_accuracy: 0.8371
Epoch 14/100
34/34 [==============================] - 0s 1ms/step - loss: 6.3884 - eval_accuracy: 0.8390
Epoch 15/100
34/34 [==============================] - 0s 1ms/step - loss: 6.9124 - eval_accuracy: 0.7760
Epoch 16/100
34/34 [==============================] 

c:\Users\82105\anaconda3\envs\project1\lib\site-packages\keras\metrics.py:257: UserWarning: Metric EvalAccuracy implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  'consistency.' % (self.__class__.__name__,))
